In [110]:
import pandas as pd
import numpy as np
from scipy import stats

In [19]:
py_df = pd.read_csv("../data/2020_sharing_data_outside.csv", low_memory=False)

In [ ]:
oecd_df = pd.read_csv(
    "../data/oecd_locations.csv", header=None, names=["abbrev", "country"]
)

In [20]:
# columns categories are separated by '.' with 1, 2 or 3 field names
# general column names
generals = [
    "age",
    "are.you.datascientist",
    "is.python.main",
    "company.size",
    "country.live",
    "employment.status",
    "first.learn.about.main.ide",
    "how.often.use.main.ide",
    "is.python.main",
    "main.purposes",
    "missing.features.main.ide",
    "nps.main.ide",
    "python.version.most",
    "python.years",
    "python2.version.most",
    "python3.version.most",
    "several.projects",
    "team.size",
    "use.python.most",
    "years.of.coding",
]


def format_field(s):
    if s in generals:
        return ("general", s)
    else:
        return s.rsplit(".", 1)


py_df.columns = pd.MultiIndex.from_tuples(
    # [(x.rsplit(".", 1) if x.count(".") >= 2 else x.split(".")) for x in py_df.columns],
    [format_field(x) for x in py_df.columns]
)
py_df = py_df[sorted(py_df.columns)]

# Questions
1. What are the most popular 10 Python IDEs?
2. Which 10 other programming languages (`other.lang`) are most commonly used by Python devs?
3. What are the 10 most common countries respondents come from?
4. According to the Python survey, what proportion of devs have each level of experience? (`python.years`)
5. Which country has the greatest number of devs with 11+ years of experience?
6. Which country has the greatest proportion of devs with 11+ years of experience?

In [24]:
# 10 most common Python IDEs
py_df[("ide", "main")].value_counts().iloc[:10]

(ide, main)
VS Code                         8010
PyCharm Professional Edition    5144
PyCharm Community Edition       3815
Vim                             2176
Sublime Text                    1201
Jupyter Notebook                1167
Atom                             784
Other                            711
Emacs                            636
Spyder                           580
Name: count, dtype: int64

In [ ]:
# 10 other programming languages are most commonly used
# each of the columns either has NaN or its programming language name, so just a count
# will figure out how many respondents answered with that language, given it skips NaN values
py_df["other.lang"].count().sort_values(ascending=False).iloc[:10]

JavaScript      16662
HTML/CSS        15469
Bash / Shell    13793
SQL             13391
C/C++           11623
Java             8109
C#               4460
PHP              4060
TypeScript       3717
Other            3592
dtype: int64

In [41]:
# 10 most common countries respondents come from
py_df[("general", "country.live")].value_counts(ascending=False).iloc[:10]

(general, country.live)
United States         3975
India                 2800
Germany               1807
China                 1155
United Kingdom        1110
France                1078
Russian Federation     935
Other country          880
Brazil                 812
Canada                 644
Name: count, dtype: int64

In [198]:
# what proportion of devs have each level of experience?
py_df[("general", "python.years")].value_counts() / py_df[
    ("general", "python.years")
].count() * 100
# The book reminds me that I can just use the normalize parameter to value_counts:
py_df[("general", "python.years")].value_counts(normalize=True)

(general, python.years)
3–5 years          0.28
Less than 1 year   0.24
1–2 years          0.22
6–10 years         0.15
11+ years          0.10
Name: proportion, dtype: float64

In [ ]:
py_df[("general", "years.of.coding")].value_counts() / py_df[
    ("general", "years.of.coding")
].count() * 100

(general, years.of.coding)
Less than 1 year   33.74
1–2 years          19.41
3–5 years          19.01
11+ years          15.76
6–10 years         12.08
Name: count, dtype: float64

In [78]:
# which country has the greatest number of devs with 11+ years of experience?
devs11_by_country = (
    py_df.loc[py_df[("general", "python.years")] == "11+ years"]
    .groupby(("general", "country.live"))
    .count()[("general", "python.years")]
    .sort_values(ascending=False)
)
devs11_by_country

(general, country.live)
United States     691
Germany           308
United Kingdom    207
France            166
Australia          94
                 ... 
Tunisia             1
Egypt               1
Nigeria             1
Pakistan            1
Viet Nam            1
Name: (general, python.years), Length: 70, dtype: int64

In [ ]:
# country with the greatest proportion of devs with 11+ years of experience
devs_by_country = py_df.groupby(("general", "country.live"))[
    [("general", "country.live")]
].count()

# devs11_by_country / devs_by_country
country_years = devs11_by_country.to_frame().join(devs_by_country)
(
    country_years[("general", "python.years")]
    / country_years[("general", "country.live")]
).sort_values(ascending=False)
# so Norway has the highest proportion of 11+ year Python devs with 26.5%

(general, country.live)
Norway        0.265432
Ireland       0.225490
Australia     0.225420
Belgium       0.225108
Slovenia      0.224490
                ...   
India         0.015714
Viet Nam      0.013158
Bangladesh    0.009174
Pakistan      0.006849
Nigeria       0.006173
Length: 70, dtype: float64

In [ ]:
# let's look at the years of coding and the main purpose for the respondents
py_df.groupby([("general", "main.purposes")])[
    [("general", "years.of.coding")]
].value_counts(normalize=True) * 100

(general, main.purposes)                     (general, years.of.coding)
Both for work and personal                   3–5 years                    23.01
                                             Less than 1 year             21.35
                                             1–2 years                    20.96
                                             11+ years                    19.50
                                             6–10 years                   15.18
For personal, educational  or side projects  Less than 1 year             66.93
                                             1–2 years                    16.64
                                             3–5 years                     8.08
                                             11+ years                     5.14
                                             6–10 years                    3.21
For work                                     Less than 1 year             22.91
                                             3–5

# Stack Overflow survey data
1. Show the average salary for different types of employment (contractors, freelancers, full time)
2. Make a pivot table indexed by country, columns are education levels, and cells contain average salary for each education level per country
3. Create this pivot table again for only countries in the OECD subset. In which country does someone with only an Associate Degree earn the most? In which of them does someone with a doctorate earn the most?
4. Remove rows from `so_df` where `LanguageHaveWorkedWith` is `NaN`
5. Remove rows from `so_df` where Python is not a commonly used language (`LanguageHaveWorkedWith`)
6. Remove rows from `so_df` where `YearsCode` is `NaN`. How many rows remain?
7. Replace the string `Less than 1 year` in `YearsCode` with `0`. Replace the string `More than 50 years` with `51`
8. Turn `YearsCode` into an integer column.
9. Create a new column called `experience` to categorise the values in `YearsCode` to:
  - Less than 1 year
  - 1-2 years
  - 3-5 years
  - 6-10 years
  - 11+ years
10. According to the Stack Overflow survey what proportion of Python developers have each level of experience?

In [171]:
# load the SO survey data
so_df = pd.read_csv("../data/so_2021_survey_results.csv")
so_df.shape[0]

83439

In [172]:
pd.options.display.float_format = "{:,.2f}".format


In [173]:
# the EdLevel values are really long, let's just keep the bit not in brackets
so_df["EdLevel"] = so_df["EdLevel"].str.split(r" \(").str[0]

In [ ]:
# show the average salary for different types of employment
(
    so_df.groupby("Employment")["ConvertedCompYearly"]
    .mean()
    .dropna()
    .sort_values(ascending=False)
    .apply("${:,.2f}".format)
)

Employment
I prefer not to say                                     $1,455,643.25
Employed full-time                                        $121,369.67
Independent contractor, freelancer, or self-employed      $107,433.97
Retired                                                    $69,533.25
Employed part-time                                         $41,136.12
Name: ConvertedCompYearly, dtype: object

In [175]:
so_df["CompZScore"] = so_df.groupby("Employment")["ConvertedCompYearly"].transform(
    lambda x: stats.zscore(x, nan_policy="omit")
)
(
    so_df[(so_df["CompZScore"] < 3) & (so_df["CompZScore"] > -3)]
    .groupby("Employment")["ConvertedCompYearly"]
    .describe()
)
# adjusting to remove values more than 3 standard deviations from the mean for each group, full timers still get
# more than other groups

,count,mean,std,min,25%,50%,75%,max
Employment,,,,,,,,
Employed full-time,"40,323.00","88,346.70","138,424.34",1.00,"28,308.00","57,072.00","98,321.00","1,608,000.00"
Employed part-time,"1,390.00","32,237.51","33,368.53",12.00,"9,129.00","22,056.00","46,424.00","320,688.00"
I prefer not to say,31.00,"43,202.32","40,039.49",68.00,"8,472.00","33,972.00","66,929.50","160,932.00"
"Independent contractor, freelancer, or self-employed","4,724.00","84,080.85","112,732.12",1.00,"24,522.00","56,875.00","108,098.00","1,395,996.00"
Retired,16.00,"69,533.25","73,898.83",100.00,"9,796.50","32,910.00","120,252.50","220,000.00"


In [176]:
# Make a pivot table indexed by country, columns are education levels, and cells contain average salary for each education level per country
so_df.pivot_table(index="Country", columns="EdLevel", values="ConvertedCompYearly")

EdLevel,Associate degree,Bachelor’s degree,Master’s degree,Other doctoral degree,Primary/elementary school,Professional degree,Secondary school,Some college/university study without earning a degree,Something else
Country,,,,,,,,,
Afghanistan,NaN,"30,288.00","10,176,704.00",NaN,NaN,NaN,100.00,NaN,NaN
Albania,NaN,"19,152.86","80,127.62",NaN,NaN,"5,298.00","19,890.00","22,884.00","128,522.00"
Algeria,NaN,"21,770.67","15,052.57",NaN,NaN,"12,912.00",NaN,"6,288.00",NaN
Andorra,NaN,"94,045.50","22,056.00","146,981.00",NaN,NaN,NaN,NaN,NaN
Angola,NaN,"31,500.00",NaN,NaN,NaN,NaN,"18,678.00","6,904.00",NaN
...,...,...,...,...,...,...,...,...,...
"Venezuela, Bolivarian Republic of...",NaN,"30,108.77","28,680.00",NaN,"7,200.00","14,833.29","10,200.00","17,720.57",NaN
Viet Nam,"7,827.00","18,463.11","50,599.80","2,592.00","10,479.00","30,000.00",NaN,"18,866.19",NaN
Yemen,NaN,"5,628.67",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# same pivot table but only include countries from the OECD data set
(
    so_df[so_df["Country"].isin(oecd_df["country"])]
    .pivot_table(index="Country", columns="EdLevel", values="ConvertedCompYearly")
    .T
)
# Finland and Israel  have their highest earners with Associate degrees
# Australia (ignoring the primary school), France, and Japan have their highest earners with doctoral degrees

Country,Australia,Austria,Belgium,Brazil,Canada,Denmark,Finland,France,Germany,Hungary,Israel,Italy,Japan
EdLevel,,,,,,,,,,,,,
Associate degree,"117,049.64","43,623.38","35,664.00","25,347.42","87,930.35","80,217.33","282,353.67","54,394.89","98,530.52","51,041.00","146,420.90","36,427.94","143,196.83"
Bachelor’s degree,"180,794.16","66,096.13","68,474.70","47,681.23","140,668.24","80,430.94","69,381.34","65,062.38","102,751.23","48,756.45","124,973.40","73,697.22","85,821.47"
Master’s degree,"106,794.32","77,645.66","88,580.41","42,056.01","144,733.35","115,751.85","74,360.98","94,585.73","110,611.98","46,771.71","128,492.71","79,132.19","130,143.19"
Other doctoral degree,"150,234.96","74,783.17","80,832.44","43,123.21","102,989.35","102,785.19","61,508.25","140,402.86","108,718.46","52,833.60","131,812.62","93,490.78","157,239.40"
Primary/elementary school,"153,327.50","86,877.00","11,342,349.25","7,880.00","73,787.83","91,056.57","77,832.00","34,181.00","74,943.67","48,100.00","100,514.57","525,992.17","44,965.00"
Professional degree,"108,725.00","38,915.33","71,000.20","25,449.67","82,953.40","114,144.89","83,016.00","78,342.27","97,330.14","9,852.00","132,336.00","117,121.37",NaN
Secondary school,"158,931.28","47,438.83","43,224.13","15,072.89","180,584.81","90,411.74","59,424.32","53,981.81","82,865.84","33,440.80","97,938.78","75,114.40","57,916.40"
Some college/university study without earning a degree,"127,063.53","53,906.05","88,006.29","39,978.02","155,090.08","85,131.35","89,255.56","55,119.45","95,737.77","37,102.89","130,049.04","81,531.72","91,163.04"
Something else,"231,987.17","45,772.53","27,035.50","20,288.71","60,795.17","95,479.86","35,024.00","42,884.04","109,333.99","28,911.00","96,142.50","32,690.22","194,514.00"


In [ ]:
# looking at the book solution, the question was actually where can you earn the most with an Associate degree or
# a doctoral degree - not in which countries were those the highest paid of the education levels
(
    so_df[so_df["Country"].isin(oecd_df["country"])]
    .pivot_table(index="Country", columns="EdLevel", values="ConvertedCompYearly")[
        "Associate degree"
    ]
    .sort_values(ascending=False)
    .iloc[:1]
)

Country
Finland   282,353.67
Name: Associate degree, dtype: float64

In [ ]:
(
    so_df[so_df["Country"].isin(oecd_df["country"])]
    .pivot_table(index="Country", columns="EdLevel", values="ConvertedCompYearly")[
        "Other doctoral degree"
    ]
    .sort_values(ascending=False)
    .iloc[:1]
)


Country
Japan   157,239.40
Name: Other doctoral degree, dtype: float64

In [178]:
# remove rows LangageWorkedWith is NaN
so_python_df = so_df.dropna(subset=["LanguageHaveWorkedWith"])
so_python_df.shape[0]

82357

In [ ]:
# remove rows in which Python is not in the LanguageHaveWorkedWith
so_python_df = so_python_df.loc[
    so_python_df["LanguageHaveWorkedWith"].str.contains("Python")
]
so_python_df.shape[0]

39792

In [184]:
# remove all rows in which YearsCode is NaN
so_python_df = so_python_df.dropna(subset=["YearsCode"])
so_python_df.shape[0]

38997

In [ ]:
# turn the YearsCode column into an integer type
def yearsCodeTransformer(s):
    if s == "Less than 1 year":
        return "0"
    elif s == "More than 50 years":
        return "51"
    else:
        return s


so_python_df["YearsCode"] = (
    so_python_df["YearsCode"].transform(yearsCodeTransformer).astype(np.int32)
)
so_python_df.shape[0]

38997

In [ ]:
so_python_df["experience"] = pd.cut(
    so_python_df["YearsCode"],
    bins=[0, 1, 2, 5, 10, float("inf")],
    labels=["Less than 1 year", "1-2 years", "3-5 years", "6-10 years", "11+ years"],
    include_lowest=True,
)
so_python_df["experience"].value_counts()

experience
11+ years           14561
6-10 years          12424
3-5 years            8678
1-2 years            1850
Less than 1 year     1484
Name: count, dtype: int64

In [ ]:
# proportion of SO users are in each experience bin
(so_python_df["experience"].value_counts() / so_python_df.shape[0]) * 100

experience
11+ years          37.34
6-10 years         31.86
3-5 years          22.25
1-2 years           4.74
Less than 1 year    3.81
Name: count, dtype: float64